In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
path_in = '/kaggle/input/customer-segmentation-tutorial-in-python/'
os.listdir(path_in)

In [ ]:
mall_data = pd.read_csv(path_in+'Mall_Customers.csv')

In [ ]:
mall_data.head()

In [ ]:
#Data Exploration
mall_data.describe()

In [ ]:
#Checking for Null Values
mall_data.isnull().sum()

In [ ]:
#Since No null values, no need to deal with missing Values

#Cheking for Outliers



In [ ]:
#Method1 :  Using Box Plot
#As you know box plot will plot the points and hence cannot be used for categorical
import seaborn as sns
sns.boxplot(x = mall_data['Annual Income (k$)'])
#Only one outlier point at 140K

In [ ]:
sns.boxplot(x = mall_data['Age'])


In [ ]:
sns.boxplot(x = mall_data['Spending Score (1-100)'])

In [ ]:
#Method 2
#Using the Z Score. If the Z score is greater than 3 or less than -3 then we consider something as an outlier



In [ ]:
mall_data.dtypes

In [ ]:
#Getting only the numeric data 
mall_data1 = mall_data._get_numeric_data()
mall_data2 = mall_data.select_dtypes(include=['float64'])

In [ ]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(mall_data1))
#print(z)

threshold = 3
print(np.where(z > 3))

#No variables which have outliers

In [ ]:
#Method 3 Interquartile Range
#IQR is difference between 75th and 25th percentiles, which is the upper and lower quartiles, Q3-Q1
Q1 = mall_data1.quantile(0.25)
Q3 = mall_data1.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
print((mall_data1 < (Q1 - 1.5*IQR)) | (mall_data1 > (Q3 + 1.5*IQR)))

In [ ]:
#Treating the outliers found via IQR

mall_data1_out = mall_data1[~((mall_data1 < (Q1 - 1.5 * IQR)) |(mall_data1 > (Q3 + 1.5 * IQR))).any(axis=1)]


In [ ]:
#result = pd.merge(mall_data1_out, mall_data['CustomerID','Gender'], on="CustomerID", how = "left")

In [ ]:
#Data Visualisations

#Distorted Plot

import matplotlib.pyplot as plt 
import plotly as py
import plotly.graph_objs as go



plt.figure(1 , figsize = (15 , 6))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    n += 1
    plt.subplot(1 , 3 , n)
    plt.subplots_adjust(hspace =0.5 , wspace = 0.5)
    sns.distplot(mall_data1_out[x] , bins = 20)
    plt.title('Distplot of {}'.format(x))
plt.show()

In [ ]:
#Categorical Variable Gender's Plot


plt.figure(1 , figsize = (15 , 5))
sns.countplot(y = 'Gender' , data = mall_data)
plt.show()

In [ ]:
#Plotting relationship between Ages, Annual Income and Spendign Score


plt.figure(1 , figsize = (15 , 7))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    for y in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
        n += 1
        plt.subplot(3 , 3 , n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
        sns.regplot(x = x , y = y , data = mall_data1_out)
        plt.ylabel(y.split()[0]+' '+y.split()[1] if len(y.split()) > 1 else y )
plt.show()

In [ ]:
#Age Vs Income relationship Scatter Plot wrt to Gender

plt.figure(1 , figsize = (15 , 6))
for gender in ['Male' , 'Female']:
    plt.scatter(x = 'Age' , y = 'Annual Income (k$)' , data = mall_data[mall_data['Gender'] == gender] ,
                s = 200 , alpha = 0.5 , label = gender)
plt.xlabel('Age'), plt.ylabel('Annual Income (k$)') 
plt.title('Age vs Annual Income w.r.t Gender')
plt.legend()
plt.show()

In [ ]:
plt.figure(1 , figsize = (15 , 6))
for gender in ['Male' , 'Female']:
    plt.scatter(x = 'Annual Income (k$)',y = 'Spending Score (1-100)' ,
                data = mall_data[mall_data['Gender'] == gender] ,s = 200 , alpha = 0.5 , label = gender)
plt.xlabel('Annual Income (k$)'), plt.ylabel('Spending Score (1-100)') 
plt.title('Annual Income vs Spending Score w.r.t Gender')
plt.legend()
plt.show()

In [ ]:
#Distribution of Age, Income according to Spending score

plt.figure(1 , figsize = (15 , 7))
n = 0 
for cols in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    n += 1 
    plt.subplot(1 , 3 , n)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
    sns.violinplot(x = cols , y = 'Gender' , data = mall_data , palette = 'vlag')
    sns.swarmplot(x = cols , y = 'Gender' , data = mall_data)
    plt.ylabel('Gender' if n == 1 else '')
    plt.title('Boxplots & Swarmplots' if n == 2 else '')
plt.show()

In [ ]:
'''Age and spending Score'''
from sklearn.cluster import KMeans
X1 = mall_data[['Age' , 'Spending Score (1-100)']].iloc[: , :].values
X1

In [ ]:
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
    algorithm.fit(X1)
    inertia.append(algorithm.inertia_)

In [ ]:
#Getting the Inertia for each of the clusters in a plot

plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

In [ ]:
#Doint it for only one set of clusyters to check
algorithm = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(X1)
labels1 = algorithm.labels_ #Tells which element belongs to which cluster
centroids1 = algorithm.cluster_centers_


In [ ]:

h = 0.02
x_min, x_max = X1[:, 0].min() - 1, X1[:, 0].max() + 1
y_min, y_max = X1[:, 1].min() - 1, X1[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

In [ ]:
#Plotting the custers(Not here we are still just doing correlation or clusters based on Age and Spending scores)
plt.figure(1 , figsize = (15 , 7) )
plt.clf()
Z = Z.reshape(xx.shape)
plt.imshow(Z , interpolation='nearest', 
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')

plt.scatter( x = 'Age' ,y = 'Spending Score (1-100)' , data = mall_data , c = labels1 , 
            s = 200 )
plt.scatter(x = centroids1[: , 0] , y =  centroids1[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Spending Score (1-100)') , plt.xlabel('Age')
plt.show()

Segmentation using Annual Income and Spending Score


In [ ]:
X2 = mall_data[['Annual Income (k$)' , 'Spending Score (1-100)']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
    algorithm.fit(X2)
    inertia.append(algorithm.inertia_)

plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

In [ ]:
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()



In [ ]:
algorithm = (KMeans(n_clusters = 5 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(X2)
labels2 = algorithm.labels_
centroids2 = algorithm.cluster_centers_

In [ ]:
h = 0.02
x_min, x_max = X2[:, 0].min() - 1, X2[:, 0].max() + 1
y_min, y_max = X2[:, 1].min() - 1, X2[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z2 = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

In [ ]:
plt.figure(1 , figsize = (15 , 7) )
plt.clf()
Z2 = Z2.reshape(xx.shape)
plt.imshow(Z2 , interpolation='nearest', 
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')

plt.scatter( x = 'Annual Income (k$)' ,y = 'Spending Score (1-100)' , data = mall_data , c = labels2 , 
            s = 200 )
plt.scatter(x = centroids2[: , 0] , y =  centroids2[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Spending Score (1-100)') , plt.xlabel('Annual Income (k$)')
plt.show()

In [ ]:
#for Visualtization purpose only here we used 2 variables, when we run the
#final code use all varibales and it has all these variables on all Axes and 
#then plot the points, those clusters the formed are the cluster of the data set

#pick the cluster with leaset inertia, which is distnace between points and its centroid. This kind of clustering can be used
#for customer segregation based on their age,spending patterns and annual income